In [1]:
from scipy.io import loadmat
import h5py
import re
import numpy as np
import validation

In [2]:
py_filepath = 'C:/Users/camil/Documents/int/inVibe/results/Riesling/LIP/2022-06-09_10-40-34/2/Riesling_LIP_2022-06-09_10-40-34_2.npy'
mat_filepath = "C:/Users/camil/Documents/int/data/openephys/2022-06-09_10-40-34/Record Node 102/experiment1/recording1/220609_TSCM_grid_Riesling.mat"

In [3]:
mat_res = validation.read_matlab_file(mat_filepath)
py_f = np.load(py_filepath,allow_pickle=True)
bhv= py_f[1]
py_f = py_f[0]
print(mat_res.keys())
print(py_f.keys())
print(bhv[0].keys())

dict_keys(['/data/NEURO/CodeNumbers', '/data/NEURO/CodeTimes', '/data/NEURO/Eyes/samples', '/data/NEURO/LFP/samples', '/data/NEURO/LFP/timestamps', '/data/NEURO/MUA', '/data/NEURO/Neuron'])
dict_keys(['times', 'block', 'clusters_id', 'clustersch', 'clustersgroup', 'clusterdepth', 'code_numbers', 'code_times', 'eyes_sample', 'lfp_sample', 'timestamps'])
dict_keys(['trial', 'AbsoluteTrialStartTime', 'Btn1', 'Btn10', 'Btn2', 'Btn3', 'Btn4', 'Btn5', 'Btn6', 'Btn7', 'Btn8', 'Btn9', 'Eye', 'Eye2', 'EyeExtra', 'Gen1', 'Gen10', 'Gen11', 'Gen12', 'Gen13', 'Gen14', 'Gen15', 'Gen16', 'Gen2', 'Gen3', 'Gen4', 'Gen5', 'Gen6', 'Gen7', 'Gen8', 'Gen9', 'Joystick', 'Joystick2', 'KeyInput', 'Mouse', 'PhotoDiode', 'SampleInterval', 'Touch', 'CodeNumbers', 'CodeTimes', 'Block', 'BlockCount', 'Condition', 'CycleRate', 'Angle', 'BackgroundColor', 'Info', 'Position1', 'Position2', 'Position3', 'Position4', 'Scale', 'Status', 'Time', 'Zorder', 'ReactionTime', 'EndTimes', 'StartTimes', 'Attribute11', 'Attribute

#### Check neurons and mua times

In [4]:
start_idx = np.where(mat_res['/data/NEURO/CodeNumbers'].reshape(1,-1)[0]==9)[0]
end_idx = np.where(mat_res['/data/NEURO/CodeNumbers'].reshape(1,-1)[0]==18)[0]
start_timestamps = mat_res['/data/NEURO/CodeTimes'][0][start_idx]
end_timestamps = mat_res['/data/NEURO/CodeTimes'][0][start_idx[1:]]
end_timestamps = np.append(end_timestamps,[mat_res['/data/NEURO/LFP/timestamps'][0][-1]])

In [5]:
validation.check_spikes(mat_res, py_f, start_timestamps, end_timestamps)

Number of neurons do match
Number of mua do match
Number of spikes do match
Spike times do match
